In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os

In [ ]:
def retrieve_contour_coords (mask_path):
    mask = cv2.imread (mask_path, cv2.IMREAD_GRAYSCALE)
    contours,_ = cv2.findContours (mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #Con esta línea indicamos que queremos sólo el contorno "padre" (RETR_EXTERNAL)y todos los puntos de este (chain_approx_none)
    coordinates_points =[]
    for contour in contours:
        for coordinates_point in contour:
            x,y = coordinates_point [0]
            coordinates_points.append ((x,y))

    print(f"Total points in contours: {len(coordinates_points)}")
    print(coordinates_points[:len(coordinates_points)])
    return coordinates_points

In [ ]:
def draw_roi (image, ccoordinates,name,laterality, view):

    # Filter coordinates based on the current index
    vertices = [(x_i, y_i) for x_i, y_i in ccoordinates]
    points = np.array(vertices, np.int32)
    points = points.reshape((-1, 1, 2))
    cv2.polylines(image, [points], isClosed=True, color=(255, 0, 255), thickness=3)
    
    plt.imshow(image, cmap = 'gray')
    plt.axis('off')
    plt.title(f"{name} | {laterality} | {view}")
    plt.show()

In [ ]:
image = cv2.imread ("image.png", cv2.IMREAD_GRAYSCALE)
name ='P_00001'
laterality = 'L'
view = 'MLO'
coordinates = retrieve_contour_coords ("./mask_path")
draw_roi (image, coordinates, name, laterality, view)


## For a full dataset

In [ ]:
def get_mask_info_for_xlxs (directory_path):
    df = pd.DataFrame(columns=["Patient", "Laterality", "View", "Label", "MaskPath"])
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(".png"):
            # Extract relevant information from the filename
            parts = re.split(r"[_-]", filename)
            patient = parts[0] + "_" + parts [1]
            laterality = parts[2]
            view = parts[3]
            label = parts[6]
            image_path = os.path.join(directory_path, filename)

            new_row = {"Patient": patient,
                       "Laterality": laterality,
                       "View": view,
                       "Label": label,
                       "MaskPath": image_path}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


    # Save the DataFrame to an XLSX file
    output_file = "CBIS_Seg.xlsx"
    df.to_excel(output_file, index=False)

In [ ]:
def assign_index (excel_file):
    df = pd.read_excel(excel_file)

    current_index = 0

    # Iterate through the rows
    for i in range(1, len(df)):
        prev_row = df.iloc[i - 1]
        curr_row = df.iloc[i]

        # Compare relevant columns
        if (prev_row["Patient"] != curr_row["Patient"] or
                prev_row["Laterality"] != curr_row["Laterality"] or
                prev_row["View"] != curr_row["View"]):
            current_index = 0
        else:
            current_index += 1

        # Assign the index value
        df.at[i, "Index"] = current_index


    df.to_excel(excel_file, index=False)

    print(f"Indexed information saved to {excel_file}")

In [ ]:
def assign_number (excel_file):
    df = pd.read_excel(excel_file)
    current_number = 1
    for i in range(1, len(df)):
        prev_row = df.iloc[i - 1]
        curr_row = df.iloc[i]

        # Compare relevant columns
        if (prev_row["Patient"] == curr_row["Patient"] and
                prev_row["Laterality"] == curr_row["Laterality"] and
                prev_row["View"] == curr_row["View"]):
            df.at[i, "Number"] = current_number
        else:
            current_number += 1
            df.at[i, "Number"] = current_number
        

    df.to_excel(excel_file, index=False)
        

In [ ]:
def contour_coords(excel_file):
    df = pd.read_excel(excel_file)
    coordinates_points_list = []  

    for i, row in df.iterrows():
        mask_path = row['MaskPath']  
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        coordinates_points = []

        for contour in contours:
            for coordinates_point in contour:
                x, y = coordinates_point[0]
                coordinates_points.append((x, y))

        coordinates_points_list.append(coordinates_points) 

    # Add the 'coordinates_points_list' to the DataFrame (e.g., as a new column)
    df['ROIPoints'] = coordinates_points_list
    df.to_excel(excel_file, index=False)

    print(f"Indexed information saved to {excel_file}")


In [ ]:
def match_image (directory_path, excel_file):
    df1 = pd.read_excel(excel_file)
    image_data = []
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(".png"):
            parts = re.split(r"[_-]", filename)
            image_patient = parts[0] + "_" + parts [1]
            image_laterality = parts[2]
            image_view = parts[3]
            image_path = os.path.join(directory_path, filename)

            image_data.append((image_patient, image_laterality, image_view, image_path))
    
    df1['ImagePath'] = ""

    for i, row in df1.iterrows():
        patient = row['Patient']
        laterality = row['Laterality']
        view = row['View']

        for image_tuple in image_data:
            if image_tuple[0] == patient and image_tuple[1] == laterality and image_tuple[2] == view:
                df1.at[i, 'ImagePath'] = image_tuple[3]
                break  # Stop searching once a match is found

    df1.to_excel(excel_file, index=False)


In [ ]:
get_mask_info_for_xlxs("masks/directory")
assign_index ("file/directory")
assign_number ("file/directory")
retrieve_contour_coords ("file/directory")
match_image ("images/directory", "file/directory")
assign_number ("file/directory")

In [ ]:
def draw_polygon_roi_from_mask(excel_file, requested_number, requested_index):
    df = pd.read_excel(excel_file)
    
    # Initialize an empty list to store all coordinates
    all_coordinates = []
    
    for i, row in df.iterrows():
        number = row['Number']
        index = row['Index']
        coordinates = row['ROIPoints']
        image_path = row['ImagePath']

        if number== requested_number  and requested_index == "all":
            patient = row['Patient']
            laterality = row['Laterality']
            view = row['View']
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # Append all coordinates to the list
            real_coordinates = eval(coordinates) # Parsing para el ROI almacenado como string, leer como lista de nuevo
            all_coordinates.append(real_coordinates)
        elif number == requested_number and index == requested_index:
            # Draw the coordinates for the specific row
            patient = row['Patient']
            laterality = row['Laterality']
            view = row['View']
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            real_coordinates = eval(coordinates)
            vertices = [(x_i, y_i) for x_i, y_i in real_coordinates]
            points = np.array(vertices, np.int32)
            points = points.reshape((-1, 1, 2))
            cv2.polylines(image, [points], isClosed=True, color=(0, 0, 0), thickness=3)

    if requested_index == "all":
        for real_coordinates in all_coordinates:
            vertices = [(x_i, y_i) for x_i, y_i in real_coordinates]
            points = np.array(vertices, np.int32)
            points = points.reshape((-1, 1, 2))
            cv2.polylines(image, [points], isClosed=True, color=(0, 0, 0), thickness=3)

        plt.imshow(image, cmap ='gray')
        plt.axis('off')
        plt.title(f"{patient} | {laterality} | {view}")
        plt.show()

In [ ]:
requested_number = 796
requested_index = "all"
draw_polygon_roi_from_mask ("file/directory", requested_number, requested_index)